# Train a Simple Regression Model

The process of training a machine learning (ML) model can be thought of as fitting a
highly parameterized function to map inputs to outputs. An ML algorithm needs to train
numerous examples of input and output pairs to accurately map an input to an output,
i. e., make a prediction. After training, the result is referred to as a trained ML model or an artifact.

This tutorial will detail how we can use [AMPL](https://github.com/ATOMScience-org/AMPL) tools to train a regression model to predict 
how much a compound will inhibit the SLC6A3 protein as measured by pKi. 
We will train a random forest model using the following inputs:

1. The curated SLC6A3 dataset from **tutorial 2**.
2. The split file generated in **tutorial 3**.
3. [RDKit](https://github.com/rdkit/rdkit) features calculated by the [AMPL](https://github.com/ATOMScience-org/AMPL) pipeline.

We will explain the use of descriptors, how to evaluate model performance,
and where the model is saved as a .tar.gz file.

> **Note** *Training a random forest model and splitting the dataset are non-deterministic. 
You will obtain a slightly different random forest model by running this tutorial each time.*

## Model Training (using already split data)

We will use the curated dataset created in **tutorial 2** and the split file 
created in **tutorial 3** to build a json file for training. We set `"previously_split": "True"`
 and set the `split_uuid`. 
Here, we will use `"split_uuid": "c35aeaab-910c-4dcf-8f9f-04b55179aa1a"` which is saved in `dataset/` as a convenience for these tutorials.

[AMPL](https://github.com/ATOMScience-org/AMPL) provides an extensive featurization module that can generate a 
variety of molecular feature types, given SMILES strings as input. 
For demonstration purposes, we choose to use RDKit features in this tutorial.

When the featurized dataset is not previously saved for SLC6A3_Ki, 
[AMPL](https://github.com/ATOMScience-org/AMPL) will create a featurized dataset and save it in a folder called `scaled_descriptors` 
as a csv file e.g. `dataset/scaled_descriptors/SLC6A3_Ki_with_rdkit_raw_descriptors.csv`

In [1]:
# importing relevant libraries
import pandas as pd
from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse

# Set up
dataset_file = 'dataset/SLC6A3_Ki_curated.csv'
odir='dataset/SLC6A3_models'

response_col = "avg_pKi"
compound_id = "compound_id"
smiles_col = "base_rdkit_smiles"
split_uuid = "c35aeaab-910c-4dcf-8f9f-04b55179aa1a"

params = {
        "verbose": "True",
        "system": "LC",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "regression",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        "featurizer": "computed_descriptors",
        "descriptor_type" : "rdkit_raw",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        "rerun": "False",
        "result_dir": odir
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

Skipped loading some Jax models, missing a dependency. No module named 'jax'
/home/apaulson/repos/AMPL_umbrella/AMPL/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)


## Model Training (Split data and train)

It is possible to split and train a model in one step. 
Here, we set `"previously_split": "False"` and not have a split_uuid parameter. 
[AMPL](https://github.com/ATOMScience-org/AMPL) splits the data by the type of split specified in the splitter parameter, 
scaffold in this example, and writes the split file in
`dataset/SLC6A3_Ki_train_valid_test_scaffold_{split_uuid}.csv.` 
After training, [AMPL](https://github.com/ATOMScience-org/AMPL) saves the model and all of its parameters as a tarball in the result_dir.

In [2]:
params = {
        "verbose": "True",
        "system": "LC",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "regression",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
    
        "previously_split": "False",
        "split_only": "False",
        "splitter": "scaffold",
        "split_valid_frac": "0.15",
        "split_test_frac": "0.15",
    
        "featurizer": "computed_descriptors",
        "descriptor_type" : "rdkit_raw",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        "rerun": "False",
        "result_dir": odir
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

/home/apaulson/repos/AMPL_umbrella/AMPL/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)


## Performance of the Model
We evaluate model performance by measuring how a accurate 
models are on validation and test sets. 
The validation set is used while optimizing the model and choosing the best
parameter settings. Then the performance on the test set is to final judge of
model performance.

AMPL has several popular metrics to evaulate regression models; 
Mean Absolute Error (MAE), Root Mean Squared Error (RMSE) and R² (R-Squared).
In our tutorials, we will use R² metric to compare our models. The best model will have the highest
R² score.

> **Note** *The model tracker client will not be supported in your environment.*

In [3]:
# Model Performance
from atomsci.ddm.pipeline import compare_models as cm
pred_df = cm.get_filesystem_perf_results(odir, pred_type='regression')

Found data for 2 models under dataset/SLC6A3_models


The pred_df dataframe has details about the model_uuid, model_path, ampl_version, model_type, features, splitter and the results for popular metrics that help evaluate the performance. Let us view the contents of the pred_df dataframe.

In [4]:
# save pred_df
import os
pred_df.to_csv(os.path.join(odir, 'pred_df.csv'))

In [5]:
# View the pred_df dataframe
pred_df

,model_uuid,model_path,ampl_version,model_type,dataset_key,features,splitter,split_strategy,split_uuid,model_score_type,...,dropouts,xgb_gamma,xgb_learning_rate,xgb_max_depth,xgb_colsample_bytree,xgb_subsample,xgb_n_estimators,xgb_min_child_weight,model_parameters_dict,feat_parameters_dict
0,8afb64d6-993e-4d8b-9072-60dcb40d2c83,dataset/SLC6A3_models/SLC6A3_Ki_curated_model_...,1.6.0,RF,/home/apaulson/repos/AMPL/atomsci/ddm/examples...,rdkit_raw,scaffold,train_valid_test,c35aeaab-910c-4dcf-8f9f-04b55179aa1a,r2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""rf_estimators"": 500, ""rf_max_depth"": null, ""...",{}
1,c9ed9de2-2f37-47cd-be4f-b5f1a2348b2f,dataset/SLC6A3_models/SLC6A3_Ki_curated_model_...,1.6.0,RF,/home/apaulson/repos/AMPL/atomsci/ddm/examples...,rdkit_raw,scaffold,train_valid_test,109e6070-7338-464f-ad8f-ce7e774506f7,r2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""rf_estimators"": 500, ""rf_max_depth"": null, ""...",{}


## Top Performing Model
To pick the top performing model, we sort the `best_valid_r2_score` column in descending order and pick the one that is maximum.

In [6]:
# Top performing model
top_model=pred_df.sort_values(by="best_valid_r2_score", ascending=False).iloc[0]
top_model

model_uuid                               8afb64d6-993e-4d8b-9072-60dcb40d2c83
model_path                  dataset/SLC6A3_models/SLC6A3_Ki_curated_model_...
ampl_version                                                            1.6.0
model_type                                                                 RF
dataset_key                 /home/apaulson/repos/AMPL/atomsci/ddm/examples...
features                                                            rdkit_raw
splitter                                                             scaffold
split_strategy                                               train_valid_test
split_uuid                               c35aeaab-910c-4dcf-8f9f-04b55179aa1a
model_score_type                                                           r2
feature_transform_type                                          normalization
model_choice_score                                                   0.489673
best_train_r2_score                                             

## Model Tarball 
The model_path or the location of the tarball where the top performing model is saved is in `top_model.model_path`.

In [8]:
# Top performing model path
top_model.model_path

'dataset/SLC6A3_models/SLC6A3_Ki_curated_model_8afb64d6-993e-4d8b-9072-60dcb40d2c83.tar.gz'

In **tutorial 5** we will learn how to use a selected model to make predictions and evaluate those predictions